# Create a 2D Surface Grain Boundary in a film

Use commensurate lattice matching algorithm to create grain boundaries in 2D materials by finding appropriate twist angles between two phases of the same material.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells.
1. Wait for the run to complete (depending on the parameters can take a few min).
1. Scroll down to view results.

## Notes

1. We perform commensurate lattice matching to find valid supercells that achieve the desired twist angle.
1. When the matching is finished, grain boundaries with angles close to the target are presented.
1. The algorithm searches for supercell matrices within specified size limits.
2. The two phases are placed next to each other in the x-direction with a gap in between.
1. Atoms on the edge of the left phase are handled to overlap with the right phase in the interfacial region.
1. For more information, see the [Introduction](Introduction.ipynb) notebook.


## 1. Prepare the Environment
### 1.1. Set up the notebook
Set the following flags to control the notebook behavior
For more information on the parameters and algorithm, refer to [Grain Boundary Builder Source](https://github.com/Exabyte-io/made/blob/35b9f318f5d667e0f5af023f3178bc4404317ab0/src/py/mat3ra/made/tools/build/grain_boundary/builders.py#L103)


In [ ]:
# Material selection
MATERIAL_INDEX = 0  # Index in the list of materials

# Grain boundary parameters
TARGET_TWIST_ANGLE = 17.9  # in degrees
INTERPHASE_GAP = 2.0  # Gap between two phases in X direction, in Angstroms
XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]] # Supercell matrix for the resulting structure

# Search algorithm parameters
MAX_REPETITION = 6  # Maximum supercell matrix element value
ANGLE_TOLERANCE = 2.5  # in degrees
RETURN_FIRST_MATCH = True  # If True, returns first solution within tolerance

# Distance tolerance for atoms to be considering too close when merging, and previous one be removed.  
# Should be less than the expected bond length
DISTANCE_TOLERANCE = 1.2  # in Angstroms

# How much of the edge region to keep for left material to overlap in the interfacial region. A fine-tuning parameter.
EDGE_INCLUSION_TOLERANCE = 1.0  # in Angstroms

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
CELL_REPETITIONS_FOR_VISUALIZATION = [3, 3, 1]

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("", "../../config.yml")

### 1.3. Load and preview input material

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

## 2. Prepare Material
### 2.1. Select and visualize initial material

In [ ]:
from utils.visualize import visualize_materials

material = materials[MATERIAL_INDEX]

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(material, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)

## 3. Generate Surface Grain Boundary
### 3.1. Set up grain boundary configuration and builder


In [ ]:
from mat3ra.made.tools.build.grain_boundary import (
    SurfaceGrainBoundaryConfiguration,
    SurfaceGrainBoundaryBuilderParameters,
    SurfaceGrainBoundaryBuilder
)

config = SurfaceGrainBoundaryConfiguration(
    film=material,
    twist_angle=TARGET_TWIST_ANGLE,
    distance_z=INTERPHASE_GAP,
    gap=INTERPHASE_GAP,
    xy_supercell_matrix=XY_SUPERCELL_MATRIX
)

params = SurfaceGrainBoundaryBuilderParameters(
    max_supercell_matrix_int=MAX_REPETITION,
    angle_tolerance=ANGLE_TOLERANCE,
    return_first_match=RETURN_FIRST_MATCH,
    edge_inclusion_tolerance=EDGE_INCLUSION_TOLERANCE,
    distance_tolerance=DISTANCE_TOLERANCE
)

builder = SurfaceGrainBoundaryBuilder(build_parameters=params)

### 3.2. Generate and analyze grain boundaries


In [ ]:
from utils.plot import plot_twisted_interface_solutions

grain_boundaries = builder.get_materials(config)

print(f"\nFound {len(grain_boundaries)} possible structures")
for i, gb in enumerate(grain_boundaries):
    actual_angle = gb.metadata.get("actual_twist_angle", "unknown")
    print(f"\nGrain Boundary {i + 1}:")
    print(f"Actual twist angle: {actual_angle}°")
    print(f"Number of atoms: {len(gb.basis.elements.ids)}")

if len(grain_boundaries) > 0:
    plot_twisted_interface_solutions(grain_boundaries)

## 4. Preview the selected grain boundary
By default, the first grain boundary is selected. You can change the selection by changing the `selected_gb` index.

In [ ]:
selected_structure = grain_boundaries[0]
actual_angle = selected_structure.metadata.get("build").get("configuration").get("actual_twist_angle")
print(f"Target angle: {TARGET_TWIST_ANGLE}°")
print(f"Actual angle: {actual_angle}°")
print(f"Number of atoms: {len(selected_structure.basis.elements.ids)}")

visualize_materials(selected_structure, repetitions=[1, 1, 1])
visualize_materials(selected_structure, repetitions=[1, 1, 1], rotation="-90x")

### 5. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_materials

set_materials(selected_structure)